In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, InputExample, evaluation, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [2]:
# Load cleaned job descriptions and resumes (provided by Person 1)
job_df = pd.read_csv('/Users/apsarasrinivasan/Downloads/job_descriptions.csv')
resume_df = pd.read_csv('/Users/apsarasrinivasan/Downloads/resumes.csv')

In [3]:
# Combine job description text
job_df['combined_text'] = job_df[['Company_Overview', 
                                  'Responsibilities', 
                                  'Required_Qualifications', 
                                  'Preferred_Qualifications']].fillna('').agg(' '.join, axis=1)

# Combine resume text
resume_df['combined_text'] = resume_df[['Professional_Summary', 
                                        'Work_Experience', 
                                        'Projects', 
                                        'Certifications', 
                                        'Education', 
                                        'Skills']].fillna('').agg(' '.join, axis=1)

In [4]:
# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
resume_embeddings = model.encode(resume_df['combined_text'].tolist(), convert_to_tensor=True)
job_embeddings = model.encode(job_df['combined_text'].tolist(), convert_to_tensor=True)

# Convert to NumPy arrays for cosine similarity
resume_embeddings_np = resume_embeddings.cpu().numpy()
job_embeddings_np = job_embeddings.cpu().numpy()

In [5]:
# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(resume_embeddings_np, job_embeddings_np)

In [6]:
# Create similarity DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=resume_df['ID'], columns=job_df['ID'])

In [7]:
# Build training dataset (positive matches only)
train_examples_all = []

for job_id in job_df['ID']:
    top_resumes = similarity_df[job_id].sort_values(ascending=False).head(3).index.tolist()
    job_text = job_df[job_df['ID'] == job_id]['combined_text'].values[0]
    for resume_id in top_resumes:
        resume_text = resume_df[resume_df['ID'] == resume_id]['combined_text'].values[0]
        train_examples_all.append(InputExample(texts=[resume_text, job_text], label=1.0))

In [8]:
# Add random negative examples
for _ in range(len(train_examples_all)):
    rand_resume = resume_df.sample(1).iloc[0]
    rand_job = job_df.sample(1).iloc[0]
    train_examples_all.append(InputExample(texts=[rand_resume['combined_text'], rand_job['combined_text']], label=0.0))

In [9]:
# Split into training and test sets
train_examples, test_examples = train_test_split(train_examples_all, test_size=0.2, random_state=42)

In [10]:
# Create DataLoader and define loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [11]:
# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=100)

# Save the fine-tuned model
model.save('/Users/apsarasrinivasan/fine_tuned_cv_model')

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1500 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1500 [00:00<?, ?it/s]

In [12]:
# Evaluate on test set
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_examples)
model.evaluate(evaluator)

np.float64(0.8561226942850587)

In [13]:
# Generate top-5 ranked resumes per job
all_rankings = []

for job_id in job_df['ID']:
    ranked = similarity_df[job_id].sort_values(ascending=False).reset_index()
    ranked.columns = ['Resume_ID', 'Similarity_Score']
    ranked['Job_ID'] = job_id
    top_ranked = ranked.head(5)
    all_rankings.append(top_ranked)

combined_df = pd.concat(all_rankings, ignore_index=True)
combined_df = combined_df[['Job_ID', 'Resume_ID', 'Similarity_Score']]